In [201]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import time
import matplotlib.colors as mc


In [197]:
def get_color_dict():
    k = ['highlight','retrofit_cat','new_cat','white','black',
         'colorbar_r4','colorbar_r3','colorbar_r2','colorbar_r1',
         'colorbar_b1','colorbar_b2','colorbar_b3','colorbar_b4']
    c = ["#e2c45e","#6f462e","#c2cb7e","#e9ede9","#1c221c","#da381f","#ee6a35","#fbac6d","#edd5a9","#81a8a0","#238695","#1a698a","#062551"]
    return dict(zip(k,c))

def is_retrofit(built_year,plant_year):
    if plant_year==built_year:
        return 'New Build'
    else:
        return 'Retrofit'

In [230]:
rgba = []

for key in get_color_dict().keys():
    if 'colorbar' in key:
        tple = mc.to_rgb(get_color_dict()[key])
         tple = (tple[0]*25)
        # + (0.25,)
        tple = (round(tple[0], 2),round(tple[1], 2),round(tple[2], 2),tple[3])
        rgba.append('rgba' + str(tple))

rgba

['rgba(0.8549019607843137, 0.2196078431372549, 0.12156862745098039, 0.25)',
 'rgba(0.9333333333333333, 0.41568627450980394, 0.20784313725490197, 0.25)',
 'rgba(0.984313725490196, 0.6745098039215687, 0.42745098039215684, 0.25)',
 'rgba(0.9294117647058824, 0.8352941176470589, 0.6627450980392157, 0.25)',
 'rgba(0.5058823529411764, 0.6588235294117647, 0.6274509803921569, 0.25)',
 'rgba(0.13725490196078433, 0.5254901960784314, 0.5843137254901961, 0.25)',
 'rgba(0.10196078431372549, 0.4117647058823529, 0.5411764705882353, 0.25)',
 'rgba(0.023529411764705882, 0.1450980392156863, 0.3176470588235294, 0.25)']

In [212]:
a = (1, 2, 3)
b = a + (4, 5, 6)  # (1, 2, 3, 4, 5, 6)
c = b[1:]  # (2, 3, 4, 5, 6)
c

(2, 3, 4, 5, 6)

In [182]:
data = pd.read_csv('/Users/jmccarty/GitHub/uipvapp/assets/database_records.csv')
print(len(data))
data.head(5)

294


,Project Name,Project Source,Project Type,Project Plant Year,Project Built Year,Project Country,Project Latitude,Project Longitude,Project Function,Project Description,...,Module Transparency,Module Uniques,Module Cost,Fake Modules?,Data Source(s),Image Name,Economic FIT,Economic DR,Economic Price Change,Note
0,16 Sime Road,BIPV SG,New Construction,NaN,NaN,NaN,1.336177,103.819272,Single Family Residential,It is known to be unusual for residential unit...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Raeber MFH Thermal,Swiss BIPV Competence Center,New Construction,2010.0,2010.0,Switzerland,47.130633,7.219001,Multi Family Residential,NaN,...,opaque,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4 Times Square,NaN,NaN,1999.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,68 Jalan Bumbong,BIPV SG,New Construction,NaN,NaN,NaN,1.415561,103.759343,Single Family Residential,"Along the roads of Jalan Bumbong, there is a r...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7 & 9 Tampines Grande,BIPV SG,New Construction,2009.0,2009.0,NaN,1.355524,103.941531,Single Family Residential,The Tampinese Grande buildings are known for t...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Stacked Histogram

In [116]:

plot_data = data[['Project Built Year','Project Plant Year']].dropna().copy()
plot_data['Project Type'] = plot_data.apply(lambda x: is_retrofit(x['Project Built Year'],x['Project Plant Year']),axis=1)
new_build_data = plot_data[plot_data['Project Type']=='New Build']['Project Plant Year'].astype(int)
retrofit_data = plot_data[plot_data['Project Type']=='Retrofit']['Project Plant Year'].astype(int)

years = np.arange(plot_data['Project Plant Year'].min(),time.localtime().tm_year)


years_df = pd.DataFrame(pd.Series(years).astype(int).rename('index'))
new = new_build_data.value_counts().reset_index().set_index('index').rename(columns={'Project Plant Year':'newbuild'})
retro = retrofit_data.value_counts().reset_index().set_index('index').rename(columns={'Project Plant Year':'retrofit'})

years_df = years_df.set_index('index').join(new)
years_df = years_df.join(retro).fillna(0)
years_df = years_df.astype(int)


In [161]:
years = years_df.index.tolist()
newbuild = years_df.newbuild.tolist()
retro = years_df.retrofit.tolist()

fig = go.Figure(data=[
    go.Bar(name='New Consutrction', x=years, y=newbuild,
           marker_color='#' + get_color_dict()['new_cat']),
    go.Bar(name='Retrofit', x=years, y=retro,
           marker_color='#' + get_color_dict()['retrofit_cat'])
])
# Change the bar mode



fig.update_layout(
    barmode='stack',
    title={"text":"BIPV Projects by Year and Type",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
    xaxis_title="Year",
    yaxis_title="Project Count",
    legend_title="Legend Title",
    font=dict(
        family="Urbanist",
        size=18,
        color='#' + get_color_dict()['black']
    )
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.98,
    xanchor="left",
    x=0.01
))

fig.show()

#### Yield (kWh x kW)

In [191]:

cols = ['Project Name', #hover
        'Project Plant Year', #retrofit or new build
        'Project Built Year', #retrofit or new build 
        'Project Country', #hover
        'Project Function', #hover
        'System Building Element', #hover
        'System Rating', 
        'System Specific Yield', 
        'System Generation Measured',
        'System Generation Simulated',
        'System Cost', 
        'System Cost Intensity',
        'System Coverage']
plot_data = data[cols].copy()
plot_data['Project Type'] = plot_data.apply(lambda x: is_retrofit(x['Project Built Year'],x['Project Plant Year']),axis=1)
plot_data.dropna(subset=['Project Function','System Rating','System Specific Yield'],inplace=True)
plot_data['System Specific Yield'] = plot_data['System Specific Yield'].astype(float)
plot_data = plot_data[plot_data['System Specific Yield']!=0]
plot_data['System Generation Measured'].fillna(0,inplace=True)
plot_data['System Generation Simulated'].fillna(0,inplace=True)
plot_data['System Generation'] = plot_data[['System Generation Measured', 'System Generation Simulated']].values.max(1)


In [195]:
new = plot_data[plot_data['Project Type']=='New Build']
retro = plot_data[plot_data['Project Type']=='Retrofit']

x_new = new['System Rating']
y_new = new['System Generation']

x_retro = retro['System Rating']
y_retro = retro['System Generation']

fig = go.Figure(data=[
    go.Scatter(name='New Build',x=x_new, y=y_new, mode='markers',
           marker_color='#' + get_color_dict()['new_cat']),
    go.Scatter(name='Retrofit',x=x_retro, y=y_retro, mode='markers',
           marker_color='#' + get_color_dict()['retrofit_cat'])])


fig.update_layout(
    title={"text":"BIPV Projects by Generation and Capacity",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
    xaxis_title="System Capacity (kW)",
    yaxis_title="Annual Generation (kWh/year)",
    legend_title="Legend Title",
    font=dict(
        family="Urbanist",
        size=18,
        color='#' + get_color_dict()['black']
    )
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.98,
    xanchor="left",
    x=0.01
))

fig.show()